# Wordle Solver

We'll start with a dumb solver and work out way up to something nicer.

Let's start with some dependencies we need to make this work.

In [1]:
%use kandy(0.4.4)

First we need to "model" the game. The game state is a list of guesses, where each guess is a five-letter word and each letter is colored yellow, green or red depending on whether the letter is in the word, in the right place, or not in the word at all. We'll represent this as a list of tuples, where each tuple is a letter and a color.

Sealed classes seem like a nice way to model colored letters, let's also write some `toString` implementations so we can print them out nicely.

In [2]:
sealed class Letter {
    abstract val char: Char

    protected fun toString(color: String) = "<td style=\"background-color: $color\">$char</td>"

    data class Yellow(override val char: Char) : Letter() {
        override fun toString() = toString("yellow")
    }

    data class Green(override val char: Char) : Letter() {
        override fun toString() = toString("green")
    }

    data class Red(override val char: Char) : Letter() {
        override fun toString() = toString("red")
    }
}

Let's test it out

In [3]:
HTML("<table><tr>${listOf(Letter.Green('a'), Letter.Yellow('b'), Letter.Red('c')).joinToString("")}</tr></table>")

a,b,c


Excellent, now let's model the game state


In [4]:
data class Game(val guesses: List<List<Letter>>) {
    init {
        require(guesses.size <= 5) { "Game must have 5 guesses at most" }
        require(guesses.all { it.size == 5 }) { "Each guess must have 5 letters" }
    }

    override fun toString() = "<table>${guesses.joinToString("") { "<tr>${it.joinToString("")}</tr>" }}</table>"
}

Let's test it out

In [5]:
HTML(
    Game(
        listOf(
            listOf(Letter.Green('a'), Letter.Yellow('b'), Letter.Red('c'), Letter.Green('d'), Letter.Yellow('e')),
            listOf(Letter.Green('v'), Letter.Yellow('w'), Letter.Red('x'), Letter.Green('y'), Letter.Yellow('z'))
        )
    ).toString()
)

a,b,c,d,e
v,w,x,y,z


Let's define a "guesser" abstraction and a dummy implementation so we can write a game driver around it.

In [6]:
typealias Guesser = (Game) -> String

val staticGuesser: Guesser = { "abcde" }

Let's write a game driver that takes a guesser and runs a game until it's over.

In [7]:
fun playGame(actualWord: String, guesser: Guesser): Game {
    var game = Game(emptyList())
    var solved = false
    while (game.guesses.size < 5 && !solved) {
        val guess = guesser(game)
        check(guess.length == 5) { "Guess must be 5 letters" }

        val letters = guess.mapIndexed { index, char ->
            when {
                char == actualWord[index] -> Letter.Green(char)
                char in actualWord -> Letter.Yellow(char)
                else -> Letter.Red(char)
            }
        }

        game = game.copy(guesses = game.guesses + listOf(letters))
        solved = letters.all { it is Letter.Green }
    }

    return game
}

Let's play a game with our dummy guesser and make it win

In [8]:
HTML(playGame("abcde", staticGuesser).toString())

a,b,c,d,e


Now make it lose

In [9]:
HTML(playGame("awcye", staticGuesser).toString())

a,b,c,d,e
a,b,c,d,e
a,b,c,d,e
a,b,c,d,e
a,b,c,d,e


To write better guessers we need to know the dictionary of valid words. Let's load that up.

In [10]:
val dictionary = java.io.File("/usr/share/dict/words")
    .readLines()
    .filter { it.length == 5 }
    .filter { it.all { char -> char in 'a'..'z' } }
    .filter { it.toSet().size == 5 }

println("loaded ${dictionary.size} words")


loaded 5497 words


We also need to get more useful information out of the game state, it's not easy to work with nested lists all the time, so let's write some helper functions to make it easier to work with.


In [11]:
// just the words
fun Game.words() = guesses.map { it.joinToString("") { letter -> letter.char.toString() } }

// all red letters
fun Game.redLetters() = guesses.flatMap { it.filterIsInstance<Letter.Red>() }
    .map { it.char }
    .toSet()

// all green letters but we need to know the positions too
fun Game.greenLetters() = guesses
    .map { it.withIndex() }
    .flatMap { it.filter { (_, letter) -> letter is Letter.Green } }
    .distinctBy { it.value.char }
    .associate { it.value.char to it.index }

// all yellow letters, except those that turned out green later.
// we also need positions for these but remember that a yellow letter can be in multiple positions
fun Game.yellowLetters(): Map<Char, Set<Int>> = guesses
    .map { it.withIndex() }
    .flatMap { it.filter { (_, letter) -> letter is Letter.Yellow } }
    .groupBy { it.value.char }
    .mapValues { it.value.map { it.index }.toSet() }
    .minus(greenLetters().keys)

// let's see if it works
fun gameFunctionsTest() {
    val game = Game(
        listOf(
            listOf(Letter.Green('a'), Letter.Yellow('b'), Letter.Red('c'), Letter.Green('d'), Letter.Yellow('e')),
            listOf(Letter.Green('a'), Letter.Green('e'), Letter.Yellow('b'), Letter.Green('d'), Letter.Red('f'))
        )
    )
    println("red: ${game.redLetters()}")
    println("green: ${game.greenLetters()}")
    println("yellow: ${game.yellowLetters()}")
}

gameFunctionsTest()

red: [c, f]
green: {a=0, d=3, e=1}
yellow: {b=[1, 2]}


With that we can now write a better guesser that finds the first word in the dictionary that has none of the red letters, all the green letters at the same positions, and all the yellow letters in any position. For this iteration we will not take into account the positions of the yellow letters.


In [12]:
fun simpleFirstMatchGuesser(dictionary: List<String>, debug: Boolean = false): Guesser = { game ->
    val redLetters = game.redLetters()
    val greenLetters = game.greenLetters()
    val yellowLetters = game.yellowLetters()
    val usedWords = game.words()

    fun Sequence<String>.f(name: String, fn: (String) -> Boolean) = filter {
        val result = fn(it)
        if (debug) {
            println("? $name: $it -> $result")
        }
        result
    }

    dictionary
        .asSequence()
        .f("used") { word -> word !in usedWords }
        .f("red($redLetters)") { word -> word.none { it in redLetters } }
        .f("green($greenLetters)") { word -> greenLetters.all { (char, index) -> word[index] == char } }
        .f("yellow($yellowLetters)") { word -> yellowLetters.all { (char, _) -> char in word } }
        .first()
}

// let's test it out
fun testSimpleFirstMatchGuesser() {
    val simpleDict = listOf("abcde", "xxxxx", "afcge", "yyyyy", "agcfe", "zzzzz")
    val guesser = simpleFirstMatchGuesser(simpleDict, debug = true)

    // shortcuts!
    fun g(char: Char) = Letter.Green(char)
    fun y(char: Char) = Letter.Yellow(char)
    fun r(char: Char) = Letter.Red(char)
    val (a, b, c, d, e) = ('a'..'e').toList()
    val (f, g) = ('f'..'g').toList()

    fun game(vararg letters: Letter) = Game(letters.toList().chunked(5))

    val firstGuess = guesser(game())
    println("> first guess: $firstGuess")
    check(firstGuess == "abcde") { "first guess should be abcde, was $firstGuess" }

    val secondGuess = guesser(
        game(
            g(a), r(b), g(c), r(d), g(e)
        )
    )
    println("> second guess: $secondGuess")
    check(secondGuess == "afcge") { "second guess should be afcge, was $secondGuess" }

    val thirdGuess = guesser(
        game(
            g(a), r(b), g(c), r(d), y(e),
            g(a), y(f), g(c), y(g), g(e)
        )
    )

    println("> third guess: $thirdGuess")
    check(thirdGuess == "agcfe") { "third guess should be agcfe, was $thirdGuess" }
}

testSimpleFirstMatchGuesser()


? used: abcde -> true
? red([]): abcde -> true
? green({}): abcde -> true
? yellow({}): abcde -> true
> first guess: abcde
? used: abcde -> false
? used: xxxxx -> true
? red([b, d]): xxxxx -> true
? green({a=0, c=2, e=4}): xxxxx -> false
? used: afcge -> true
? red([b, d]): afcge -> true
? green({a=0, c=2, e=4}): afcge -> true
? yellow({}): afcge -> true
> second guess: afcge
? used: abcde -> false
? used: xxxxx -> true
? red([b, d]): xxxxx -> true
? green({a=0, c=2, e=4}): xxxxx -> false
? used: afcge -> false
? used: yyyyy -> true
? red([b, d]): yyyyy -> true
? green({a=0, c=2, e=4}): yyyyy -> false
? used: agcfe -> true
? red([b, d]): agcfe -> true
? green({a=0, c=2, e=4}): agcfe -> true
? yellow({f=[1], g=[3]}): agcfe -> true
> third guess: agcfe


Now let's play a real game with our guesser. Here's a word it should be able to guess:

In [13]:
HTML(playGame("sauce", simpleFirstMatchGuesser(dictionary)).toString())

a,b,h,o,r
a,c,e,d,y
a,c,u,t,e
c,a,u,s,e
s,a,u,c,e


Sweet, so now we have a guesser that at least makes an attempt to win the game, but we don't know _how_ good it is. Let's write a benchmark function that plays a game a bunch of times and collects statistics about the number of guesses it took to win. In order to make comparisons between guessers easier we will also make sure that the same words are used in each benchmark run.


In [16]:
import java.math.BigDecimal
import java.math.RoundingMode
import kotlin.random.Random

val random = Random(0)
val benchmarkWords = dictionary.shuffled(random).take(100)

sealed class Outcome {
    object Lost : Outcome()
    data class Won(val guesses: Int) : Outcome()
}

fun Game.outcome() = if (guesses.last().all { it is Letter.Green }) {
    Outcome.Won(guesses.size)
} else {
    Outcome.Lost
}

data class Statistics(
    val percentWon: BigDecimal,
    val guessFrequency: Map<Int, BigDecimal>
)

fun benchmark(guesser: Guesser, words: List<String> = benchmarkWords, log: Boolean = false): Statistics {
    val outcomes = words.map { word ->
        val outcome = playGame(word, guesser).outcome()
        if (log) {
            if (outcome is Outcome.Won) {
                println("$word: won in ${outcome.guesses} guesses")
            } else {
                println("$word: lost")
            }
        }
        outcome
    }

    val total = outcomes.size
    val numWon = outcomes.count { it is Outcome.Won }
    val percentWon = numWon.toDouble() / total * 100

    val guessFrequency = outcomes.filterIsInstance<Outcome.Won>()
        .map { it.guesses }
        .groupingBy { it }
        .eachCount()
        .mapValues { it.value.toDouble() / total * 100 }
        .toSortedMap()

    fun Double.round() = BigDecimal(this).setScale(2, RoundingMode.HALF_EVEN)

    return Statistics(percentWon.round(), guessFrequency.mapValues { it.value.round() })
}

Right, now let's benchmark the simple guesser.


In [17]:
val simpleGuesserStats = benchmark(simpleFirstMatchGuesser(dictionary))

In [18]:
println("Simple Guesser won ${simpleGuesserStats.percentWon}% of games!")
println("Guess frequency:")
simpleGuesserStats.guessFrequency.forEach { (guesses, percent) ->
    println("$guesses: $percent%")
}

Simple Guesser won 55.00% of games!
Guess frequency:
2: 1.00%
3: 9.00%
4: 17.00%
5: 28.00%


In [19]:
plot {
    bars {
        x(simpleGuesserStats.guessFrequency.keys)
        y(simpleGuesserStats.guessFrequency.values)
    }
}

1.5
 
 
 
 
 
 
 
 
 2.0
 
 
 
 
 
 
 
 
 2.5
 
 
 
 
 
 
 
 
 3.0
 
 
 
 
 
 
 
 
 3.5
 
 
 
 
 
 
 
 
 4.0
 
 
 
 
 
 
 
 
 4.5
 
 
 
 
 
 
 
 
 5.0
 
 
 
 
 
 
 
 
 5.5
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 5
 
 
 
 
 
 
 10
 
 
 
 
 
 
 15
 
 
 
 
 
 
 20
 
 
 
 
 
 
 25
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y
 
 
 
 
 x

That's pretty cool! The simple guesser won over half the games and has a nice linear trend line on the guess frequency.

We can however, do better. Note that the simple guesser does not take into account the positions of the yellow letters. Let's write a guesser that does and benchmark it.


In [20]:
fun yellowPostionAwareGuesser(dictionary: List<String>, debug: Boolean = false): Guesser = { game ->
    val redLetters = game.redLetters()
    val greenLetters = game.greenLetters()
    val yellowLetters = game.yellowLetters()
    val usedWords = game.words()

    fun Sequence<String>.f(name: String, fn: (String) -> Boolean) = filter {
        val result = fn(it)
        if (debug) {
            println("? $name: $it -> $result")
        }
        result
    }

    dictionary
        .asSequence()
        .f("used") { word -> word !in usedWords }
        .f("red($redLetters)") { word -> word.none { it in redLetters } }
        .f("green($greenLetters)") { word -> greenLetters.all { (char, index) -> word[index] == char } }
        .f("yellow($yellowLetters)") { word ->
            yellowLetters.all { (char, indexes) ->
                char in word && indexes.all { index -> word[index] != char }
            }
        }
        .first()
}

Let's check if it can guess the same word as before:

In [21]:
HTML(playGame("sauce", yellowPostionAwareGuesser(dictionary)).toString())

a,b,h,o,r
c,a,d,e,t
l,a,n,c,e
s,a,u,c,e


Sweet, it can! Now let's benchmark it.

In [22]:
val yellowPositionAwareGuesserStats = benchmark(yellowPostionAwareGuesser(dictionary))

In [23]:
println("Yellow Position Aware Guesser won ${yellowPositionAwareGuesserStats.percentWon}% of games!")
println("Guess Frequency")
yellowPositionAwareGuesserStats.guessFrequency.forEach { (guess, frequency) ->
    println("$guess: $frequency%")
}

Yellow Position Aware Guesser won 82.00% of games!
Guess Frequency
2: 2.00%
3: 10.00%
4: 42.00%
5: 28.00%


In [24]:
plot {
    bars {
        x(yellowPositionAwareGuesserStats.guessFrequency.keys)
        y(yellowPositionAwareGuesserStats.guessFrequency.values)
    }
}

1.5
 
 
 
 
 
 
 
 
 2.0
 
 
 
 
 
 
 
 
 2.5
 
 
 
 
 
 
 
 
 3.0
 
 
 
 
 
 
 
 
 3.5
 
 
 
 
 
 
 
 
 4.0
 
 
 
 
 
 
 
 
 4.5
 
 
 
 
 
 
 
 
 5.0
 
 
 
 
 
 
 
 
 5.5
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 5
 
 
 
 
 
 
 10
 
 
 
 
 
 
 15
 
 
 
 
 
 
 20
 
 
 
 
 
 
 25
 
 
 
 
 
 
 30
 
 
 
 
 
 
 35
 
 
 
 
 
 
 40
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y
 
 
 
 
 x

That is a significant improvement! The yellow position aware guesser wins over 80% of games and has a much nicer guess frequency distribution: in fact it wins most games on or before the 4th guess.

That said, we can still do better. You'll notice that both guessers start with the same first word, which happens to be the first word in the dictionary. We also know that some letter occur more frequently than others in the English language. Let'Surely if we prioritized words that contain the most common letters we would be able to guess the word faster. Let's write a guesser that does that and benchmark it. First let's examine the frequency of letters in the dictionary.


In [25]:
val letterFrequency = dictionary
    .flatMap { it.toList() }
    .groupingBy { it }
    .eachCount()
    .mapValues { it.value.toDouble() / dictionary.size * 100 }
    .toSortedMap()

plot {
    bars {
        x(letterFrequency.keys)
        y(letterFrequency.values)
    }
}


a
 
 
 
 
 
 
 
 
 b
 
 
 
 
 
 
 
 
 c
 
 
 
 
 
 
 
 
 d
 
 
 
 
 
 
 
 
 e
 
 
 
 
 
 
 
 
 f
 
 
 
 
 
 
 
 
 g
 
 
 
 
 
 
 
 
 h
 
 
 
 
 
 
 
 
 i
 
 
 
 
 
 
 
 
 j
 
 
 
 
 
 
 
 
 k
 
 
 
 
 
 
 
 
 l
 
 
 
 
 
 
 
 
 m
 
 
 
 
 
 
 
 
 n
 
 
 
 
 
 
 
 
 o
 
 
 
 
 
 
 
 
 p
 
 
 
 
 
 
 
 
 q
 
 
 
 
 
 
 
 
 r
 
 
 
 
 
 
 
 
 s
 
 
 
 
 
 
 
 
 t
 
 
 
 
 
 
 
 
 u
 
 
 
 
 
 
 
 
 v
 
 
 
 
 
 
 
 
 w
 
 
 
 
 
 
 
 
 x
 
 
 
 
 
 
 
 
 y
 
 
 
 
 
 
 
 
 z
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 10
 
 
 
 
 
 
 20
 
 
 
 
 
 
 30
 
 
 
 
 
 
 40
 
 
 
 
 
 
 50
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y
 
 
 
 
 x

In [26]:
fun letterFrequencyGuesser(dictionary: List<String>, debug: Boolean = false): Guesser = { game ->
    val redLetters = game.redLetters()
    val greenLetters = game.greenLetters()
    val yellowLetters = game.yellowLetters()
    val usedWords = game.words()

    fun Sequence<String>.f(name: String, fn: (String) -> Boolean) = filter {
        val result = fn(it)
        if (debug) {
            println("? $name: $it -> $result")
        }
        result
    }

    dictionary
        .asSequence()
        .f("used") { word -> word !in usedWords }
        .f("red($redLetters)") { word -> word.none { it in redLetters } }
        .f("green($greenLetters)") { word -> greenLetters.all { (char, index) -> word[index] == char } }
        .f("yellow($yellowLetters)") { word ->
            yellowLetters.all { (char, indexes) ->
                char in word && indexes.all { index -> word[index] != char }
            }
        }
        .sortedByDescending { word -> word.sumOf { letterFrequency[it] ?: 0.0 } }
        .first()
}

// let's try the same sample word as before
HTML(playGame("sauce", letterFrequencyGuesser(dictionary)).toString())


a,r,i,t,e
s,l,a,n,e
s,a,u,c,e


Three guesses is definitely an improvement over the previous two guessers. Let's benchmark it.


In [27]:
val letterFrequencyGuesserStats = benchmark(letterFrequencyGuesser(dictionary))
println("Letter Frequency Guesser won ${letterFrequencyGuesserStats.percentWon}% of games!")
println("Guess Frequency:")
letterFrequencyGuesserStats.guessFrequency.forEach { (guess, frequency) ->
    println("$guess: $frequency%")
}

plot{
    bars {
        x(letterFrequencyGuesserStats.guessFrequency.keys)
        y(letterFrequencyGuesserStats.guessFrequency.values)
    }
}


Letter Frequency Guesser won 91.00% of games!
Guess Frequency:
2: 2.00%
3: 31.00%
4: 36.00%
5: 22.00%


1.5
 
 
 
 
 
 
 
 
 2.0
 
 
 
 
 
 
 
 
 2.5
 
 
 
 
 
 
 
 
 3.0
 
 
 
 
 
 
 
 
 3.5
 
 
 
 
 
 
 
 
 4.0
 
 
 
 
 
 
 
 
 4.5
 
 
 
 
 
 
 
 
 5.0
 
 
 
 
 
 
 
 
 5.5
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0
 
 
 
 
 
 
 5
 
 
 
 
 
 
 10
 
 
 
 
 
 
 15
 
 
 
 
 
 
 20
 
 
 
 
 
 
 25
 
 
 
 
 
 
 30
 
 
 
 
 
 
 35
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y
 
 
 
 
 x

That's a nice improvement! The letter frequency guesser wins over 90% of games and over third of them in three guesses or fewer!

Can we do better? I think so. Notice how all the guessers above, once they find green or yellow letters, they only select words that match those letters. Consider the case where you guessed `light` on the first attempt and got a red `l` followed by 4 greens. There are so many words that end in `ight` e.g. `bight`, `right`, `eight`, `wight`, `sight` and more. If you only guessed words ending with `ight` you'd probably run out of chances before you find the one. You're far better off guessing a word like `brews` because if any of those letters match, then you know exactly which word it is.

In a sense the guesser has to operate in two modes: start in an _exploration_ mode where the goal is to find as many green and yellow letters as possible, then switch to a _confirmation_ mode where the goal is to confirm the word. Firstly, it's hard to define how _exploration_ works, but we can define _confirmation_ quite easily: that's what the guessers above are doing. Secondly, we need to define when to switch from _exploration_ to _confirmation_ mode.

I might come back to that some other day, this will have to do for now. Would love to hear your thoughts on this.
